In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("EmployeeTable").getOrCreate()

# Define Schema
emp_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("emp_name", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("manager_id", IntegerType(), True),
    StructField("emp_age", IntegerType(), True),
    StructField("dep_id", IntegerType(), True),
    StructField("dep_name", StringType(), True),
    StructField("gender", StringType(), True)
])

# Define Data
emp_data = [
    (1, 'Ankit', 14300, 4, 39, 100, 'Analytics', 'Female'),
    (2, 'Mohit', 14000, 5, 48, 200, 'IT', 'Male'),
    (3, 'Vikas', 12100, 4, 37, 100, 'Analytics', 'Female'),
    (4, 'Rohit', 7260, 2, 16, 100, 'Analytics', 'Female'),
    (5, 'Mudit', 15000, 6, 55, 200, 'IT', 'Male'),
    (6, 'Agam', 15600, 2, 14, 200, 'IT', 'Male'),
    (7, 'Sanjay', 12000, 2, 13, 200, 'IT', 'Male'),
    (8, 'Ashish', 7200, 2, 12, 200, 'IT', 'Male'),
    (9, 'Mukesh', 7000, 6, 51, 300, 'HR', 'Male'),
    (10, 'Rakesh', 8000, 6, 50, 300, 'HR', 'Male'),
    (11, 'Akhil', 4000, 1, 31, 500, 'Ops', 'Male')
]

# Create DataFrame
emp_df = spark.createDataFrame(emp_data, schema=emp_schema)
emp_df.createOrReplaceTempView("Employees")

# Show DataFrame
emp_df.show()
from pyspark.sql.functions import *
from pyspark.sql.window import Window


+------+--------+------+----------+-------+------+---------+------+
|emp_id|emp_name|salary|manager_id|emp_age|dep_id| dep_name|gender|
+------+--------+------+----------+-------+------+---------+------+
|     1|   Ankit| 14300|         4|     39|   100|Analytics|Female|
|     2|   Mohit| 14000|         5|     48|   200|       IT|  Male|
|     3|   Vikas| 12100|         4|     37|   100|Analytics|Female|
|     4|   Rohit|  7260|         2|     16|   100|Analytics|Female|
|     5|   Mudit| 15000|         6|     55|   200|       IT|  Male|
|     6|    Agam| 15600|         2|     14|   200|       IT|  Male|
|     7|  Sanjay| 12000|         2|     13|   200|       IT|  Male|
|     8|  Ashish|  7200|         2|     12|   200|       IT|  Male|
|     9|  Mukesh|  7000|         6|     51|   300|       HR|  Male|
|    10|  Rakesh|  8000|         6|     50|   300|       HR|  Male|
|    11|   Akhil|  4000|         1|     31|   500|      Ops|  Male|
+------+--------+------+----------+-------+-----

In [21]:
spark.sql("""
with cte as (select * from
(select *,dense_rank() over (partition by dep_name order by salary desc) dr,
row_number() over (partition by dep_name order by dep_id) rn from Employees) p
where dr = 3), 
cte2 as (select min(salary) over (partition by dep_name order by salary) from Employees where rn<3)

select * from cte
"""
).show()

AnalysisException: unresolved operator 'Filter ('rn < 3);
'WithCTE
:- CTERelationDef 21, false
:  +- SubqueryAlias cte
:     +- Project [emp_id#0, emp_name#1, salary#2, manager_id#3, emp_age#4, dep_id#5, dep_name#6, gender#7, dr#742, rn#743]
:        +- Filter (dr#742 = 3)
:           +- SubqueryAlias p
:              +- Project [emp_id#0, emp_name#1, salary#2, manager_id#3, emp_age#4, dep_id#5, dep_name#6, gender#7, dr#742, rn#743]
:                 +- Project [emp_id#0, emp_name#1, salary#2, manager_id#3, emp_age#4, dep_id#5, dep_name#6, gender#7, dr#742, rn#743, dr#742, rn#743]
:                    +- Window [row_number() windowspecdefinition(dep_name#6, dep_id#5 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rn#743], [dep_name#6], [dep_id#5 ASC NULLS FIRST]
:                       +- Window [dense_rank(salary#2) windowspecdefinition(dep_name#6, salary#2 DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) 